
# Beyond CFT: Attacks on the Network and Convergence 


So far we showed how gossip can withstand network imperfection. But what if the attacker deliberately splits and attacks the network?  


Beyond regular crashes peer can behave in various ways violating the protocol: hide transactions, send bogus data, create Sybil entities etc.
The goal of a blockchain system is to withstand against a powerful adversary. 

To ensure that message will be seen by the peer, once the peer is back online it must fetch the data from the neighboring peers. But what if the neighboring nodes are malicious and will censor certain transactions?   

In the next notebooks we will cover techniques that help to detect/prevent malicious behaviour.



# Malicious gossip agent 

One of the goal of a blockchain system is to record transaction in a 'hard-to-tamper' way.

How can you achieve that in p2p settings?  
It is common in databases and blockchains to use cryptography to verify the integrities of the transactions.

Let's first create a malicious agent that will change the data of received transactions to split the network.  


In [ ]:
# Initialize the experiment:
import networkx as nx
import p2psimpy as p2p
import warnings
warnings.filterwarnings('ignore')

# Load the previous experiment configurations
exper = p2p.BaseSimulation.load_experiment(expr_dir='crash_gossip')

Locations, topology, peer_services, serv_impl = exper


## Define malicious agents 
Let's assign first malicious nodes, we will assign randomly: 

In [ ]:
# Change peer to a malicious 
from itertools import groupby
from random import sample

frac_malicious_nodes = 0.3 # 30 % of malicious nodes


def assign_malicious_peers(topology, mal_frac):
    type_dict = nx.get_node_attributes(topology, 'type')
    inv_type_dict = {k: {j for j, _ in list(v)}
                                for k, v in groupby(type_dict.items(), lambda x: x[1])}
    mal_nodes = sample(list(inv_type_dict['peer']), 
                       int(frac_malicious_nodes * len(inv_type_dict['peer'])))
    for b in mal_nodes:
        type_dict[b] = 'malicious'
        
    nx.set_node_attributes(topology, type_dict, 'type')
    
assign_malicious_peers(topology, frac_malicious_nodes)

## Define malicious services 

We will inherit a malicious gossip service that will relay the gossip message to one half of the network and the other half a tempered message (with different data). 



In [ ]:
from p2psimpy.messages import *
from p2psimpy.consts import TEMPERED

class MaliciousGossipService(p2p.GossipService):
    
    
    def handle_message(self, msg):
        # Store the original message localy 
        self.peer.store('msg_time', msg.id, self.peer.env.now)
        self.peer.store('msg_data', msg.id, msg.data)

        if msg.ttl > 0:
            # Rely message further, modify the message
            exclude_peers = {msg.sender} | self.exclude_peers
            
            # Send the original message to one half of the network, 
            selected = self.peer.gossip(GossipMessage(self.peer, msg.id, msg.data, msg.ttl-1,
                                                      pre_task=msg.pre_task, post_task=msg.post_task), 
                                        self.fanout//2, 
                                        except_peers=exclude_peers, 
                                        except_type=self.exclude_types)
            # Change the message and send it to the other half
            new_data = TEMPERED
            exclude_peers = exclude_peers | set(selected)
            self.peer.gossip(GossipMessage(self.peer, msg.id, new_data, msg.ttl-1, 
                                           pre_task=msg.pre_task, post_task=msg.post_task), 
                             self.fanout//2, 
                             except_peers=exclude_peers, 
                             except_type=self.exclude_types)

##  Add malicious type and services 

We deliberately keep malicious nodes uncrashable. 


In [ ]:
gossip_config = peer_services['peer'].service_map['RangedPullGossipService']
serv_impl['RangedPullGossipService'] = p2p.GossipService



peer_services['malicious'] = p2p.PeerType(peer_services['peer'].config,
                                      {p2p.BaseConnectionManager:None,
                                       MaliciousGossipService: gossip_config}
                                     )

## Run simulation 

Let's see how malicious agents together with crashing nodes affect the message dissemination. 

In [ ]:
serv_impl

In [ ]:
from p2psimpy.messages import GossipMessage

In [ ]:
# Init Graph
sim = p2p.BaseSimulation(Locations, topology, peer_services, serv_impl)
sim.run(3_200)

# Analyze the storage data




## Message data

Let's see how this fraction of malicious nodes affected the network. 
We compare the received message with the original message, we report `True` if the message wasn't tampered and False otherwise. 



In [ ]:
import pandas as pd

def message_data(sim, peer_id, storage_name):
    store = sim.peers[peer_id].storage[storage_name].txs
    for msg_id, tx in store.items():
        client_id, msg_num = msg_id.split('_')
        client_tx = sim.peers[int(client_id)].storage[storage_name].txs[msg_id]
        yield (int(msg_num), tx.data == client_tx.data)
        
def get_gossip_table(sim, storage_name, func):
    return pd.DataFrame({k: dict(func(sim, k, storage_name)) 
                         for k in set(sim.types_peers['peer'])}).sort_index()

    
df = get_gossip_table(sim, 'msg_data', message_data)
df

In [ ]:
df[df==False].count()

malicious nodes managed to trick some peers into accepting wrong data! As peers will write 'first-seen' value, adversary once having advantage over the network can perfectly split the network. 

How to deal with this?

# Signing messages

First of all, messages themselves must be verified on their **integrity** and **authenticity**. 
[Digital signatures](https://en.wikipedia.org/wiki/Digital_signature) are perfect match for this. That is why you see that they used in  any blockchain system. 

We will modify the code to simulate the signed messages. We will not use actual crytpographic protocol for that, for the simulation we only care about two things: 
- It takes time to verify and sign messages. 
- Peers should store and forward only valid messages. 



## Simulating digital signatures 

We will show an example by building a crypto validator for 456 bits [EdDSA (Ed448)](https://en.wikipedia.org/wiki/EdDSA) (one of the most popular digital signatures in the wild).
On a regular laptop it takes usually less than 1 millisecond to verify a signature. Let's take the near worse case.  


This is not real. 

We will integrate a verification task into the message itself. 
Peer before triggering other services will first run the `pre_task`.  

Since the message is first created by MessageProducer we will add a task in the configuration.



In [ ]:
from p2psimpy.consts import TEMPERED
from p2psimpy.config import Config, Func, Dist


conf = peer_services['client'].service_map['MessageProducer']


def validate_task(msg, peer):
    gen_dist = Dist('norm', (1, 0.2)) # time it takes to verify the message

    yield peer.env.timeout(gen_dist.get())
    if msg.data == TEMPERED:
        # You can decide what to do in this case.
        return False
    return True


class MsgConfig(Config):
    pre_task = Func(validate_task)
    init_ttl = conf.init_ttl if conf else 3
    
    
peer_services['client'].service_map['MessageProducer'] = MsgConfig

In [ ]:
# Run the simulation with a modificed message producer 

sim2 = p2p.BaseSimulation(Locations, topology, peer_services, serv_impl)
sim2.run(3_200)


In [ ]:
df = get_gossip_table(sim2, 'msg_data', message_data)
df

In [ ]:
sim2.peers[12].storage['msg_data'].txs

Now malicious nodes cannot change the message. They need to explore other attack strategies!
The malicious nodes can delay messages, hide them, freeriding in a gossip (only listening). 
Together with network attack this can create a dangerous combination. 

For example, if an honest node is surrounded by malicious nodes (all network connections are with malicious nodes) that will hide certain transactions. As a result, peer will not receive crucial transactions that might affect it's decision making process. This attack is also called **Eclipse attack**.   

In reality, almost nothing stops one malicious node from running multiple instances and poision the whole network. Such attack is called **Sybil Attack**. 




### Exercises


- Explore the limits of the gossip protocol. What is the maximum malicious node a protocol can tolerate? 
- Try to eclipse attack some peer, make sure he doesn't get any message, or one specific message (censor)? 

